In [104]:
import pandas as pd
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score,confusion_matrix
import optuna

In [11]:
df = pd.read_csv('../processed_data/train_v11.csv')
df.columns

Index(['id', 'y', 'location', 'access', 'layout', 'age', 'direction', 'area',
       'floor', 'bath_toilet',
       ...
       'kitchen_16', '23ku_countall', 'area_num_countall', 'age_countall',
       'floor_countall', 'max_floor_countall', 'layout_countall',
       'direction_countall', 'facilities_countall',
       'contract_period_countall'],
      dtype='object', length=123)

In [12]:
df[df['high_price_flag']==1]

,id,y,location,access,layout,age,direction,area,floor,bath_toilet,...,kitchen_16,23ku_countall,area_num_countall,age_countall,floor_countall,max_floor_countall,layout_countall,direction_countall,facilities_countall,contract_period_countall
171,172,415000,東京都渋谷区恵比寿４丁目,山手線\t恵比寿駅\t徒歩8分\t\t山手線\t目黒駅\t徒歩14分\t\t日比谷線\t広尾...,2LDK,25年0ヶ月,北,75.96m2,3,バス・トイレ別／\tシャワー／\t追焚機能／\t浴室乾燥機\t／\t温水洗浄便座／\t洗面台独立,...,0,1978,2,78,10410,2101,5130,3767,2,1
258,259,468000,東京都中央区勝どき６丁目,都営大江戸線\t勝どき駅\t徒歩5分\t\t有楽町線\t月島駅\t徒歩14分\t\t日比谷線...,2LDK,11年7ヶ月,南西,96.34m2,30,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t追焚機能／\t浴室乾燥機\t...,...,0,2532,4,383,22,148,5130,6122,5,1
274,275,650000,東京都品川区上大崎３丁目1-30,山手線\t目黒駅\t徒歩2分\t\t東急目黒線\t不動前駅\t徒歩12分\t\t南北線\t白...,3LDK,2年1ヶ月,北東,79.91m2,30,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t追焚機能／\t浴室乾燥機\t...,...,0,2544,1,197,22,20,2240,2667,1,1
350,351,402000,東京都目黒区碑文谷２丁目,東急東横線\t学芸大学駅\t徒歩12分,3LDK+S(納戸),16年5ヶ月,東,118.38m2,5,バス・トイレ別／\t追焚機能／\t浴室乾燥機,...,0,1941,6,292,3959,2866,133,9943,4,1
666,667,505000,東京都中央区月島１丁目5-1,都営大江戸線\t月島駅\t徒歩1分\t\t都営大江戸線\t勝どき駅\t徒歩11分\t\t東西...,3LDK,4年1ヶ月,南東,111.65m2,53,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t追焚機能／\t浴室乾燥機\t...,...,0,2532,5,133,5,34,2240,6150,152,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30904,30909,455000,東京都中央区勝どき６丁目,都営大江戸線\t勝どき駅\t徒歩5分\t\t有楽町線\t月島駅\t徒歩13分\t\t都営大江...,3LDK+S(納戸),11年7ヶ月,南東,93.88m2,38,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t追焚機能／\t浴室乾燥機\t...,...,0,2532,8,383,19,148,133,6150,14,1
30953,30958,418000,東京都千代田区麹町１丁目,半蔵門線\t半蔵門駅\t徒歩1分\t\t有楽町線\t麹町駅\t徒歩6分\t\t総武線・中央線...,1LDK,13年7ヶ月,南,68.11m2,9,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t追焚機能／\t浴室乾燥機\t...,...,0,867,2,180,1526,2200,9702,17695,10,1
30980,30985,410000,東京都新宿区新宿６丁目27-29,都営大江戸線\t東新宿駅\t徒歩2分\t\t副都心線\t東新宿駅\t徒歩2分\t\t都営新宿...,1LDK,7年7ヶ月,西,71.99m2,5,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t追焚機能／\t浴室乾燥機\t...,...,0,2500,3,116,3959,53,9702,8086,1609,1
31010,31015,430000,東京都渋谷区代々木１丁目,山手線\t代々木駅\t徒歩5分\t\t小田急小田原線\t南新宿駅\t徒歩6分\t\t副都心線...,1LDK,1年11ヶ月,北西,62.32m2,6,専用バス／\t専用トイレ／\tバス・トイレ別／\t追焚機能／\t浴室乾燥機\t／\t温水洗浄...,...,0,1978,5,222,2793,2705,9702,2742,1,1


In [35]:
use_col = df.columns

un_use_col = ['id','y','log_y','high_price_flag','prob','label',
                'location', 'access', 'layout', 'age', 'direction', 'area','floor', 'bath_toilet', 'kitchen',
                'broadcast_com', 'facilities','parking', 'enviroment', 'structure', 'contract_period',
                'walk_time','23ku',
                 'area_num_countall','floor_countall','room_num_countall','facilities_countall','age_countall','area_num_countall',
            ]
use_col = [c for c in use_col if c not in un_use_col]

In [121]:
X = df.loc[:,use_col]
y = df.loc[:,'high_price_flag']
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0,test_size=0.5)
print(len(X_train),len(y_train),len(X_test),len(y_test))

15733 15733 15733 15733


In [128]:
def objective(trial):

    learning_rate = trial.suggest_uniform('learning_rate', 0, 1.0)
#     num_leaves = trial.suggest_int('num_leaves', 10, 2**6)
    max_depth = trial.suggest_int('max_depth', 3, 8)

    lgbm_params = {
        'task': 'train',
        # "metrics": 'xentropy',
        'boosting_type': 'gbdt',
        'objective': 'binary',
        "learning_rate": learning_rate,
#         "num_leaves": num_leaves,
        "max_depth": max_depth,
        "n_jobs": 1,
        'verbose': -1,
        "seed": 0
    }

    mdl = lgb.LGBMClassifier(**lgbm_params)
    stratifiedkfold = StratifiedKFold(n_splits=3)
    scores = cross_val_score(mdl,X_train,y_train,cv=stratifiedkfold,scoring='accuracy')
    score = np.mean(scores)

    return score

In [129]:
study = optuna.create_study()
study.optimize(objective,n_trials=50)

[I 2019-10-08 22:54:29,436] Finished trial#0 resulted in value: 0.9819497311499994. Current best value is 0.9819497311499994 with parameters: {'learning_rate': 0.33292263418473644, 'max_depth': 4}.
[I 2019-10-08 22:54:30,524] Finished trial#1 resulted in value: 0.9909744413254017. Current best value is 0.9819497311499994 with parameters: {'learning_rate': 0.33292263418473644, 'max_depth': 4}.
[I 2019-10-08 22:54:31,520] Finished trial#2 resulted in value: 0.9102578522388348. Current best value is 0.9102578522388348 with parameters: {'learning_rate': 0.7541131040735222, 'max_depth': 6}.
[I 2019-10-08 22:54:32,492] Finished trial#3 resulted in value: 0.8819817983588935. Current best value is 0.8819817983588935 with parameters: {'learning_rate': 0.9365529197376342, 'max_depth': 3}.
[I 2019-10-08 22:54:33,503] Finished trial#4 resulted in value: 0.9841738293635389. Current best value is 0.8819817983588935 with parameters: {'learning_rate': 0.9365529197376342, 'max_depth': 3}.
[I 2019-10-08

[I 2019-10-08 22:55:10,628] Finished trial#39 resulted in value: 0.9870342534882862. Current best value is 0.7863351750008379 with parameters: {'learning_rate': 0.9890151568385451, 'max_depth': 4}.
[I 2019-10-08 22:55:11,744] Finished trial#40 resulted in value: 0.991101570747772. Current best value is 0.7863351750008379 with parameters: {'learning_rate': 0.9890151568385451, 'max_depth': 4}.
[I 2019-10-08 22:55:12,785] Finished trial#41 resulted in value: 0.9785812741924546. Current best value is 0.7863351750008379 with parameters: {'learning_rate': 0.9890151568385451, 'max_depth': 4}.
[I 2019-10-08 22:55:14,169] Finished trial#42 resulted in value: 0.9952964053149747. Current best value is 0.7863351750008379 with parameters: {'learning_rate': 0.9890151568385451, 'max_depth': 4}.
[I 2019-10-08 22:55:15,227] Finished trial#43 resulted in value: 0.9883046022414561. Current best value is 0.7863351750008379 with parameters: {'learning_rate': 0.9890151568385451, 'max_depth': 4}.
[I 2019-10-

In [130]:
mdl = lgb.LGBMClassifier(**study.best_params)
mdl.fit(X_train,y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.9890151568385451,
               max_depth=4, min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=31,
               objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
               silent=True, subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0)

In [131]:
pred_test = mdl.predict(X_test)
pred_test

array([0, 0, 0, ..., 0, 0, 0])

In [132]:
for i in pred_test[:,1]:
    if 0.2<i and i!=1:
        print(i)

IndexError: too many indices for array

In [133]:
confusion_matrix(y_test,pred_test)

array([[13354,  2253],
       [  121,     5]])

In [148]:
df[df['high_price_flag']==1].groupby(by='23ku').describe()['23ku_mean_std']

,count,mean,std,min,25%,50%,75%,max
23ku,,,,,,,,
世田谷区,8.0,-0.270601,0.000000e+00,-0.270601,-0.270601,-0.270601,-0.270601,-0.270601
中央区,21.0,1.684829,2.275280e-16,1.684829,1.684829,1.684829,1.684829,1.684829
北区,1.0,-0.686553,NaN,-0.686553,-0.686553,-0.686553,-0.686553,-0.686553
千代田区,22.0,1.737814,0.000000e+00,1.737814,1.737814,1.737814,1.737814,1.737814
台東区,2.0,0.132821,0.000000e+00,0.132821,0.132821,0.132821,0.132821,0.132821
品川区,11.0,0.044923,7.277573e-18,0.044923,0.044923,0.044923,0.044923,0.044923
大田区,1.0,-0.468736,NaN,-0.468736,-0.468736,-0.468736,-0.468736,-0.468736
文京区,5.0,0.160099,0.000000e+00,0.160099,0.160099,0.160099,0.160099,0.160099
新宿区,17.0,0.218723,0.000000e+00,0.218723,0.218723,0.218723,0.218723,0.218723


In [141]:
df.columns

Index(['id', 'y', 'location', 'access', 'layout', 'age', 'direction', 'area',
       'floor', 'bath_toilet',
       ...
       'kitchen_16', '23ku_countall', 'area_num_countall', 'age_countall',
       'floor_countall', 'max_floor_countall', 'layout_countall',
       'direction_countall', 'facilities_countall',
       'contract_period_countall'],
      dtype='object', length=123)

In [149]:
len(df[df['23ku']=='港区'])

1512